# Menghindari gempa berada di dua atau lebih katalog yang berbeda

Jalankan file ini di Openquake yang sudah diinstal di komputer masing-masing.

atau:

Jalankan file ini di jupyter dengan cara mengubah kernel ke `openquake`. Jika kernel `openquake` belum ada, lakukan langkah-langkah berikut ini:

Openquake dapat diunduh di sini:
https://downloads.openquake.org/pkgs/windows/oq-engine/OpenQuake_Engine_3.11.5-1.exe

Install beberapa tambahan packages di openquake:

(Asumsi lokasi instalasi openquake di sini: `C:\Program Files\Openquake Engine\python3.6\`, 
jika tidak maka ubah lokasi tersebut sesuai dengan komputer masing-masing)

1. Buka Command Prompt
2. Install `utm`: `"C:\Program Files\Openquake Engine\python3.6\python.exe" -m pip install utm`
3. Install `jupyter`: `"C:\Program Files\Openquake Engine\python3.6\python.exe" -m pip install jupyter`
4. Daftarkan ke dalam kernel (opsional): `python -m ipykernel install --name "openquake" --display-name "openquake"`


Untuk uji coba, jalankan salah satu baris di bawah ini:

`"C:\Program Files\Openquake Engine\python3.6\Scripts\jupyter.exe" notebook`

atau

`"C:\Program Files\Openquake Engine\python3.6\Scripts\jupyter-notebook.exe"`

## 1. Import packages

Hal ini memastikan apakah python yang digunakan sudah menyediakan packages yang dibutuhkan.

In [1]:
from openquake.hmtk.parsers.catalogue import *
from openquake.hmtk.plotting.seismicity.catalogue_plots import *
from openquake.hmtk.seismicity.declusterer.dec_gardner_knopoff import *
from openquake.hmtk.seismicity.declusterer.distance_time_windows import *
from openquake.hmtk.seismicity.selector import *
from openquake.hazardlib.geo.polygon import Polygon as PolyOQ
from openquake.hazardlib.geo.point import Point as PointOQ
from openquake.hazardlib.geo.line import Line as LineOQ
from openquake.hmtk.seismicity.completeness.comp_stepp_1971 import Stepp1971
from openquake.hmtk.seismicity.occurrence.b_maximum_likelihood import BMaxLikelihood
from openquake.hmtk.plotting.seismicity.completeness.plot_stepp_1972 import create_stepp_plot
from openquake.hmtk.seismicity.smoothing.smoothed_seismicity import SmoothedSeismicity
from openquake.hmtk.seismicity.smoothing.kernels.isotropic_gaussian import IsotropicGaussian
from openquake.hazardlib.geo.surface.complex_fault import ComplexFaultSurface
from shapely.geometry import *
from shapely.ops import *
import os
import utm
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import pickle
from custom_openquake import *

## 2. Tambahan

### 2.2 Variabel

In [2]:
# folder penyimpanan
dir_figs = os.path.join(os.getcwd(), 'figs')
os.makedirs(dir_figs, exist_ok=True)

## 3. Katalog gempa

In [14]:
# membuka data pkl
catalogue_area_faults = open_pkl("tmp_OQ004_catalogue_area_faults.pkl")
catalogue_megathrusts = open_pkl("tmp_OQ004_catalogue_megathrusts.pkl")
catalogue_shallow_backgrounds = open_pkl("tmp_OQ004_catalogue_shallow_backgrounds.pkl")
catalogue_deep_backgrounds, range_list = open_pkl("tmp_OQ004_catalogue_deep_backgrounds.pkl")

catalogue_sumatera_fault = catalogue_area_faults['merged'][0]
catalogue_mentawai_fault = catalogue_area_faults['individual'][0]
catalogue_megathrust_1 = catalogue_megathrusts[0]
catalogue_megathrust_2 = catalogue_megathrusts[1]
catalogue_shallow_background_1 = catalogue_shallow_backgrounds[0]
catalogue_deep_background_1_1 = catalogue_deep_backgrounds[0][0]
catalogue_deep_background_1_2 = catalogue_deep_backgrounds[0][1]
catalogue_deep_background_1_3 = catalogue_deep_backgrounds[0][2]

## 4. Filter area faults dari data megathrusts

In [18]:
###
catalogue_sumatera_fault_filt_1 = remove_events_in_A_from_B(catalogue_sumatera_fault, 
                                                            catalogue_megathrust_1)

catalogue_sumatera_fault_filt_2 = remove_events_in_A_from_B(catalogue_sumatera_fault_filt_1, 
                                                            catalogue_megathrust_2)

###
catalogue_mentawai_fault_filt_1 = remove_events_in_A_from_B(catalogue_mentawai_fault, 
                                                            catalogue_megathrust_1)

catalogue_mentawai_fault_filt_2 = remove_events_in_A_from_B(catalogue_mentawai_fault_filt_1, 
                                                            catalogue_megathrust_2)

###
catalogue_shallow_background_1_filt_1 = remove_events_in_A_from_B(catalogue_shallow_background_1, 
                                                            catalogue_megathrust_1)

catalogue_shallow_background_1_filt_2 = remove_events_in_A_from_B(catalogue_shallow_background_1_filt_1, 
                                                            catalogue_megathrust_2)

catalogue_shallow_background_1_filt_3 = remove_events_in_A_from_B(catalogue_shallow_background_1_filt_2, 
                                                            catalogue_sumatera_fault_filt_2)

catalogue_shallow_background_1_filt_4 = remove_events_in_A_from_B(catalogue_shallow_background_1_filt_3, 
                                                            catalogue_mentawai_fault_filt_2)

###
catalogue_deep_background_1_1_filt_1 = remove_events_in_A_from_B(catalogue_deep_background_1_1, 
                                                            catalogue_megathrust_1)

catalogue_deep_background_1_1_filt_2 = remove_events_in_A_from_B(catalogue_deep_background_1_1_filt_1, 
                                                            catalogue_megathrust_2)

###
catalogue_deep_background_1_2_filt_1 = remove_events_in_A_from_B(catalogue_deep_background_1_2, 
                                                            catalogue_megathrust_1)

catalogue_deep_background_1_2_filt_2 = remove_events_in_A_from_B(catalogue_deep_background_1_2_filt_1, 
                                                            catalogue_megathrust_2)

catalogue_deep_background_1_2_filt_3 = remove_events_in_A_from_B(catalogue_deep_background_1_2_filt_2, 
                                                            catalogue_deep_background_1_1_filt_2)

###
catalogue_deep_background_1_3_filt_1 = remove_events_in_A_from_B(catalogue_deep_background_1_3, 
                                                            catalogue_megathrust_1)

catalogue_deep_background_1_3_filt_2 = remove_events_in_A_from_B(catalogue_deep_background_1_3_filt_1, 
                                                            catalogue_megathrust_2)

catalogue_deep_background_1_3_filt_3 = remove_events_in_A_from_B(catalogue_deep_background_1_3_filt_2, 
                                                            catalogue_deep_background_1_2_filt_3)

jumlah gempa yang dihapus:  0
jumlah gempa yang dihapus:  0
jumlah gempa yang dihapus:  2
jumlah gempa yang dihapus:  2
jumlah gempa yang dihapus:  0
jumlah gempa yang dihapus:  1
jumlah gempa yang dihapus:  265
jumlah gempa yang dihapus:  77
jumlah gempa yang dihapus:  113
jumlah gempa yang dihapus:  108
jumlah gempa yang dihapus:  0
jumlah gempa yang dihapus:  0
jumlah gempa yang dihapus:  0
jumlah gempa yang dihapus:  0
jumlah gempa yang dihapus:  0
jumlah gempa yang dihapus:  0


In [19]:
catalogue_to_pkl(catalogue_sumatera_fault_filt_2, "dict_catalogue_sumatera_fault_filt_2.pkl")
catalogue_to_pkl(catalogue_mentawai_fault_filt_2, "dict_catalogue_mentawai_fault_filt_2.pkl")
catalogue_to_pkl(catalogue_shallow_background_1_filt_4, "dict_catalogue_shallow_background_1_filt_4.pkl")
catalogue_to_pkl(catalogue_deep_background_1_1_filt_2, "dict_catalogue_deep_background_1_1_filt_2.pkl")
catalogue_to_pkl(catalogue_deep_background_1_2_filt_3, "dict_catalogue_deep_background_1_2_filt_3.pkl")
catalogue_to_pkl(catalogue_deep_background_1_3_filt_3, "dict_catalogue_deep_background_1_3_filt_3.pkl")

variable_to_pkl(
    (
        catalogue_sumatera_fault_filt_2,
        catalogue_mentawai_fault_filt_2,
        catalogue_megathrust_1,
        catalogue_megathrust_2,
        catalogue_shallow_background_1_filt_4,
        catalogue_deep_background_1_1_filt_2,
        catalogue_deep_background_1_2_filt_3,
        catalogue_deep_background_1_3_filt_3
    ),
    "tmp_OQ004_catalogues_filtered.pkl"
)